In [1]:
import json
import os
import ase.io

from ase.io                       import write
from ase.io.vasp                  import read_vasp
from ase.build                    import bulk, sort
from ase.calculators.emt          import EMT
from hiphive.structure_generation import generate_mc_rattled_structures

Read structures containing displacements and forces

In [2]:
#root_path = f'data/CrN-riccardo-slightly-distorted/PM/SC_2x2x2_Vol_76.69'
#root_path = f'data/CrN-riccardo-slightly-distorted/PM/SC_2x2x2_Vol_76.31'
#root_path = f'data/CrN-riccardo-slightly-distorted/PM/SC_2x2x2_Vol_75.93'
root_path = f'data/CrN-riccardo-slightly-distorted/PM/SC_2x2x2_Vol_75.56'

# Parameters
n_structures     = 500
rattle_std       = 0.03  # 0.03
minimum_distance = 2.3
cell_size        = [2, 2, 2]

# Create and save as a dictionary
setup_variables = {
    'root_path':        root_path,
    'n_structures':     n_structures,
    'rattle_std':       rattle_std,
    'minimum_distance': minimum_distance,
    'cell_size':        cell_size
}

# Write the dictionary to the file in JSON format
with open(f'{root_path}/structure_generation_variables.json', 'w') as json_file:
    json.dump(setup_variables, json_file)

In [3]:
i = 0
while True:
    try:
        # Read POSCAR file
        prim = read_vasp(file=f'{setup_variables["root_path"]}/POSCAR')
        atoms_ideal = prim.repeat(cell_size)

        # Generate the structures
        structures = generate_mc_rattled_structures(atoms_ideal, n_structures, rattle_std, minimum_distance,
                                                   seed=i)
        
        print("No exception occurred. Exiting loop.")
        break  # Exit the loop if no exception occurred
    except Exception as e:
        print(f"Exception {i} occurred: {e}")
        i += 1
        rattle_std *= 0.9
        # Optionally, you can add a delay before retrying
        # time.sleep(1)

Exception 0 occurred: Exceeded the maximum number of attempts for atom 21
Exception 1 occurred: Exceeded the maximum number of attempts for atom 2
Exception 2 occurred: Exceeded the maximum number of attempts for atom 0
Exception 3 occurred: Exceeded the maximum number of attempts for atom 46
Exception 4 occurred: Exceeded the maximum number of attempts for atom 1
Exception 5 occurred: Exceeded the maximum number of attempts for atom 37
Exception 6 occurred: Exceeded the maximum number of attempts for atom 27
Exception 7 occurred: Exceeded the maximum number of attempts for atom 56
Exception 8 occurred: Exceeded the maximum number of attempts for atom 58
Exception 9 occurred: Exceeded the maximum number of attempts for atom 31
Exception 10 occurred: Exceeded the maximum number of attempts for atom 7
Exception 11 occurred: Exceeded the maximum number of attempts for atom 11
Exception 12 occurred: Exceeded the maximum number of attempts for atom 55
Exception 13 occurred: Exceeded the max

KeyboardInterrupt: 

In [5]:
rattle_std

0.0029543127065508345

In [ ]:
# Read POSCAR file
prim = read_vasp(file=f'{setup_variables["root_path"]}/POSCAR')
atoms_ideal = prim.repeat(cell_size)

# Generate the structures
structures = generate_mc_rattled_structures(atoms_ideal, n_structures, rattle_std, minimum_distance,
                                           seed=1234)

In [ ]:
for idx in range(n_structures):
    # Sort the chemical species
    structure = sort(structures[idx])
    
    # Create disp folder
    idx_path = f'{setup_variables["root_path"]}/disp-{idx}'
    os.mkdir(idx_path)
    
    # Write to POSCAR
    write(f'{idx_path}/POSCAR', structure, format='vasp')